In [626]:
import pandas as pd
import json

Inception v3 trained on balanced dataset, focal loss $\gamma = 2$, 5 epochs, 100 batch size, Adam lr=0.0005

In [627]:
path = "inception_v3/results/fl/"

In [628]:
df = pd.read_csv(path+"avg_metrics.csv")

In [629]:
# Function to parse indices string into list of ints
def parse_json_column(json_str):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return None

# Apply the parsing function to the 'data' column
df['image_idxs'] = df['image_idxs'].apply(parse_json_column)

In [630]:
df

,image_idxs,accuracy,precision,specificity,sensibility,IoU,dice_coeff,f1_score
0,"[0, 1, 2]",0.923582,0.944153,0.803675,0.955514,0.904632,0.949778,0.867521
1,"[3, 4, 5]",0.876938,0.901001,0.668477,0.934825,0.847833,0.917488,0.761862
2,"[6, 7, 8]",0.958776,0.959478,0.379743,0.996992,0.956734,0.977843,0.504840
3,"[9, 10, 11]",0.925365,0.927591,0.586084,0.988546,0.917742,0.957096,0.718273
4,"[12, 13, 14]",0.944898,0.958782,0.576420,0.980369,0.940861,0.969433,0.714726
5,"[15, 16, 17]",0.877045,0.890821,0.667746,0.926476,0.834710,0.908125,0.751503
6,"[18, 19, 20]",0.958472,0.966655,0.743838,0.986787,0.954289,0.976567,0.839826
7,"[21, 22, 23]",0.565221,0.938074,0.592196,0.489018,0.465990,0.623366,0.598168
8,"[24, 25, 26]",0.866663,0.931240,0.177983,0.918663,0.852784,0.919621,0.212807
9,"[27, 28, 29]",0.852006,0.982441,0.925630,0.835340,0.823073,0.902938,0.953106


In [631]:
metrics_df = df.drop(columns=['image_idxs'])

In [632]:
cols = metrics_df.columns
n_samples, n_features = metrics_df.shape

In [633]:
list(cols)

['accuracy',
 'precision',
 'specificity',
 'sensibility',
 'IoU',
 'dice_coeff',
 'f1_score']

In [634]:
means = metrics_df.mean()
means

accuracy       0.874897
precision      0.940024
specificity    0.612179
sensibility    0.901253
IoU            0.849865
dice_coeff     0.910225
f1_score       0.692263
dtype: float64

In [635]:
stds = metrics_df.std()/n_samples
stds

accuracy       0.011563
precision      0.002874
specificity    0.021172
sensibility    0.015263
IoU            0.014396
dice_coeff     0.010474
f1_score       0.021185
dtype: float64

In [636]:
n_decimals = 4
cross_val_data = [f"{means.iloc[i]:0.{n_decimals}f} +- {stds.iloc[i]:0.{n_decimals}f}" for i in range(n_features)]

In [637]:
cross_val_metrics_df = pd.DataFrame([cross_val_data], columns=tuple(cols))
cross_val_metrics_df.loc[0]

accuracy       0.8749 +- 0.0116
precision      0.9400 +- 0.0029
specificity    0.6122 +- 0.0212
sensibility    0.9013 +- 0.0153
IoU            0.8499 +- 0.0144
dice_coeff     0.9102 +- 0.0105
f1_score       0.6923 +- 0.0212
Name: 0, dtype: object

In [638]:
cross_val_metrics_df.to_csv(path+"cross_val_metrics.csv", index=False, columns=tuple(cols))